# Introducción a BERT con **BETO** (el BERT en español)

https://github.com/dccuchile/beto


## DESCARGAMOS EL MODELO

El modelo está en PyTorch y por tanto es compatible con la librería Transformers de HuggingFace 🤗.
Desafortunadamente el modelo no está en el hub de Transformers para cargarlo de forma directa.

In [ ]:
# First install the library and download the models from github

!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

     |████████████████████████████████| 2.2MB 19.3MB/s 
     |████████████████████████████████| 3.3MB 54.0MB/s 
     |████████████████████████████████| 870kB 51.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f8814f7a96e64af75c02918ee9eb0d7942914e972d12ff83438b39302e562476
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
--2021-04-09 10:50:00--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz’

pytorch_weights.tar 100%[===================>] 390.88M  9.42MB/s    in 51s     

2021-04-09 10:50:52 (7.72 MB/s) - ‘pytorch_weights.tar.gz

## CARGAMOS EL MODELO CON LA TAREA "MASKED LM"

Se enmascaran algunas palabras de una sentencia y BETO devuelve las candidatas según su probabilidad.

In [ ]:
# Importamos los elementos necesarios para trabajar con los modelos

import torch
from transformers import BertForMaskedLM, BertTokenizer

In [ ]:
# Creamos el "tokenizador" y el modelo con los datos de BETO

tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("pytorch/")
model.eval()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
##Función para manejar las máscaras del modelo

def unmask(model, text):
  tokens = tokenizer.tokenize(text)
  print(tokens)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
  tokens_tensor = torch.tensor([indexed_tokens])
  
  predictions = model(tokens_tensor)[0]

  masked_indxs = [i for i in range(len(tokens)) if tokens[i]=='[MASK]']
  for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:10])
    print('MASK',i,':',predicted_token)



In [ ]:
# Probamos ahora el modelo con un problema de "palabras enmascaradas"

text = "[CLS] Para [MASK] los problemas del [MASK], el presidente debe [MASK] de inmediato. [SEP]"

unmask(model, text)

['[CLS]', 'Para', '[MASK]', 'los', 'problemas', 'del', '[MASK]', ',', 'el', 'presidente', 'debe', '[MASK]', 'de', 'inmediato', '.', '[SEP]']
MASK 0 : ['resolver', 'solucionar', 'tratar', 'arreglar', 'evitar', 'abordar', 'manejar', 'todos', 'atender', 'corregir']
MASK 1 : ['presidente', 'gobierno', 'país', 'gabinete', 'Presidente', 'partido', 'mundo', 'sistema', 'estado', 'Congreso']
MASK 2 : ['actuar', 'renunciar', 'intervenir', 'presentarse', 'llamar', 'responder', 'regresar', 'reunirse', 'hacerlo', 'retirarse']


Para mejorar la predicción de máscaras en dominios más concretos sería necesario re-entrenar el modelo con ejemplos adecuados.

## CAMBIAMOS LA TAREA, CAMBIAMOS EL "HEAD"

Para la clasificación, BERT utiliza el token especial [CLS]. También se podría hacer la clasificación directamente con los embeddings de los token de salida.

Cuando cargamos el modelo para una tarea concreta, se cambia el "head" del transformer por el de la tarea concreta. Es decir se quita la capa con el que se entrenó (última de las mostradas anteriormente), y se añade una nueva para la tarea específica.

In [ ]:
## BERT para clasificar sentencias

from transformers import BertForSequenceClassification
#import torch

model2 = BertForSequenceClassification.from_pretrained("pytorch/")


In [ ]:
inputs = tokenizer("Me siento triste", return_tensors='pt')  # Batch size 1
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model2(**input_ids, labels=labels)

outputs

SequenceClassifierOutput(loss=tensor(1.1600, grad_fn=<NllLossBackward>), logits=tensor([[ 0.0911, -0.6928]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


Este clasificador no clasifica nada porque no le hemos dicho qué tiene que clasificar. De nuevo, Re-entrenando el modelo con ejemplos específicos lograremos clasificar con buena precisión.

## Y OTRA TAREA ... BUSCAR RESPUESTAS A PREGUNTAS

In [ ]:
from transformers import BertForQuestionAnswering
#import torch

model3 = BertForQuestionAnswering.from_pretrained('pytorch/')


In [ ]:
question, text = "¿Cómo se llama el presidente?", "El presidente de la república es Juan Pérez, que fue nombrado en 1992."
inputs = tokenizer(question, text, return_tensors='pt')

#print(input_ids)
tokens_answ = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
print(tokens_answ)

#Juan Pérez
start_positions = torch.tensor([7])
end_positions = torch.tensor([9])

output = model3(**inputs, start_positions=start_positions, end_positions=end_positions)

output


['[CLS]', 'El', 'presidente', 'de', 'la', 'república', 'es', 'J', '##uan', 'Pérez', ',', 'que', 'fue', 'nombrado', 'en', '1992', '.', '[SEP]']
QuestionAnsweringModelOutput(loss=tensor(3.3530, grad_fn=<DivBackward0>), start_logits=tensor([[0.5718, 0.5167, 0.2103, 0.5927, 0.4087, 0.4196, 0.3866, 0.3108, 0.7489,
         0.5119, 0.5294, 0.7897, 0.4597, 0.3280, 0.7372, 0.3749, 0.4290, 0.1096,
         0.4879, 0.7059, 0.6806, 0.3969, 0.4912, 0.0961, 0.4025, 0.7489]],
       grad_fn=<SqueezeBackward1>), end_logits=tensor([[ 0.0274,  0.0849,  0.2804, -0.1664,  0.0802, -0.0723,  0.1448,  0.0878,
         -0.2123,  0.0080,  0.0852,  0.0074, -0.1405,  0.0426,  0.0194, -0.0530,
          0.0022, -0.0888,  0.2044,  0.1623,  0.1322,  0.1192, -0.0710, -0.2225,
         -0.1345, -0.2124]], grad_fn=<SqueezeBackward1>), hidden_states=None, attentions=None)


In [ ]:
# El segmento a extraer para la respuesta sería

start_scores = torch.argmax(output.start_logits)
end_scores   = torch.argmax(output.end_logits)

print(start_scores, end_scores)
print(tokens_answ[start_scores: end_scores + 1]) # vacío ya que el final es menor que el principio

tensor(11) tensor(2)
[]


No da respuesta porque el final está antes que el principio. El modelo de base no ha sido entrenado para esa tarea y por tanto debe ajustarse con ejemplos concretos.

**CONCLUSIÓN**: Todos estos modelos deben ser entrenados en la tarea concreta ("fine-tuning") para obtener los resultados esperados.
Para ello se recomienda utilizar una librería más simple enfocada a tareas concretas, como "SimpleTransformers"